<a href="https://colab.research.google.com/github/HoarfrostRaven/Apprentissages-Automatiques-Appliques/blob/main/TD_IBLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("rent.csv")

In [ ]:
data

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,5/18/2022,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,5/13/2022,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,5/16/2022,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,7/4/2022,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,5/9/2022,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,5/18/2022,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,5/15/2022,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,7/10/2022,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,7/6/2022,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [ ]:
data.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [ ]:
####################
# Data preparation #
####################

# Feature engineering :


def get_floor(u):
  try:
    return float(str(u).split()[0])
  except:
    return 0

def max_floor(u):
  try:
    return float(str(u).split()[-1])
  except:
    return 0

data["simple_floor"] = data["Floor"].apply(get_floor)
data["max_floor"] = data["Floor"].apply(max_floor)

# Data preparation : numerisation des variables et remplacement des valeurs manquantes

def prepare_data(data, variables, variables_num, threshold_num=0.2):
  # 1er selection  des variables
  data = data[variables+var_num]
  # remplacer des valeurs manquantes
  data = data.fillna("Missing")
  # Numérisation des variables
  data = pd.get_dummies(data, columns=variables_num)
  check_var = [u for u in data.columns if data[u].dtype=="bool"]
  for var in check_var:
    #print(data[var].sum())
    if data[var].sum()<threshold_num*data.shape[0]:
      #print("ok")
      del data[var]

   # Normalisation / standardisation
  #data = (data-data.mean())/data.std() # standardisation
  #data = (data-data.min())/(data.max()-data.min()) # normalisation

  return data

variables = ['Rent', 'Size', "Bathroom","simple_floor","max_floor"]
var_num = ["Area Type"]

data_prep = prepare_data(data, variables, var_num,threshold_num=0.2)
y = data_prep["Rent"]
del data_prep["Rent"]

# Data Separation


X_train, X_test, y_train, y_test = train_test_split(data_prep, y, test_size=0.2, random_state=42)






In [ ]:
X_train


,Size,Bathroom,simple_floor,max_floor,Area Type_Carpet Area,Area Type_Super Area
1995,950,2,6.0,8.0,False,True
1497,800,2,3.0,4.0,False,True
2763,545,1,2.0,3.0,False,True
1351,465,1,3.0,4.0,False,True
1862,900,2,4.0,5.0,False,True
...,...,...,...,...,...,...
4426,1500,3,1.0,2.0,True,False
466,1200,2,3.0,3.0,False,True
3092,800,2,13.0,17.0,True,False
3772,3500,3,0.0,1.0,True,False


In [ ]:
# prompt: Kmeans sur X_train

# Quel K ?

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0) # You can change n_clusters
kmeans.fit(X_train) # Selectionner 3/4 variables max
labels = kmeans.labels_
print(labels)
# You can now use the labels for further analysis or visualization
# For instance, you can add the labels to your training data:
X_train['cluster'] = labels
print(X_train.head())


In [ ]:
# prompt: Test silhouette

from sklearn.metrics import silhouette_score

# Calculate Silhouette Score
silhouette_avg = silhouette_score(X_train, labels)
print(f"Silhouette Score: {silhouette_avg}")


#### Question 1
A l'aide des deux cellules précédentes déterminer le meilleur K pour le Kmeans.


#### Question 2

Même question mais en sélectionnant au préalable au maximum 4 variables pertinentes


##### Question 3
Vérifier le pouvoir discriminant des clusters en analysant la variation d'une variable sur chacun

###### A l'aide des plus proches voisins, déterminer le prix du loyer en fonction des variables sélectionnées. Quel est le K optimal ?

In [ ]:
# Discrimination pertinent ?

In [ ]:
# Data Apprentissage



In [ ]:

# Data Estimation

In [ ]:
# Data Evaluation